### 0 - MODULES

all modules, libraries, imports and constant used in this program

In [321]:
import pandas as pd
import matplotlib.pyplot as plt

FLIGHT_DELAY_INPUT = "FlightSchedule.csv" 
TOL = 0.0001

<h3> 1 - READ FLIGHT DETAIL</h3>

This function reads the flight detail from the csv file, <br> the file is organized as:
<ol>
<li>FL_DATE: day of the flight in format YYYY-mm-dd.</li>
<li>TAIL_NUM: aircraft registration number, unique to a single aircraft.</li>
<li>UNIQUE_CARRIER: flight carrier id.</li>
<li>FL_NUM: number of the flight</li>
<li>ORIGIN: departure airport code.</li>
<li>DEST: destination airport code.</li>
<li>CRS_DEP_TIME: scheduled departure time (local time: HHMM) shown in the carriers’ Computerized
Reservations Systems (CRS)</li>
<li>DEP_TIME: actual departure time (local time: HHMM)</li>
<li>DEP_DELAY: overall delay at departure. Difference in minutes (floating point number) between scheduled and actual departure time. Early departures set to 0.</li>
<li>CRS_ARR_TIME: scheduled arrival time (local time: HHMM) shown in the carriers’ Computerized
Reservations Systems (CRS)</li>
<li>ARR_TIME: actual arrival time (local time: HHMM)</li>
<li>ARR_DELAY: overall delay. Difference in minutes (floating point number) between scheduled and
actual arrival time. Early arrivals show negative numbers.</li>
<li>CARRIER_DELAY: delay in minutes (floating point number) caused by the carrier</li>
<li>WEATHER_DELAY: delay in minutes (floating point number) caused by the weather.</li>
<li>NAS_DELAY: delay in minutes (floating point number) caused by the National Air System (NAS).</li>
<li>SECURITY_DELAY: delay in minutes (floating point number) caused by the security</li>
<li>LATE_AIRCRAFT_DELAY: delay in minutes (floating point number) caused by the aircraft</li>
<li>There are some other fields in this dataset</li>
</ol>
Returns the pandas dataFrame filtering out the last column

In [322]:
import numpy as np
def readFileInputFlight(file:str)->pd.DataFrame:
    return pd.read_csv(file).iloc[:, :-1]

### 2 - ANSWER THE FOLLOWING QUESTION

Use the info() and describe() methods to analyze how your records are distributed. <br>
Before continuing, try to answer the following questions:
<ol>
<li>which type does each column have?</li>
<li>are there any missing values?</li>
<li>how many unique carriers are present?</li>
<li>how many unique airports are present?</li>
<li>from which time interval data were collected?</li>
</ol>

In [323]:
def answerQuestion(df:pd.DataFrame)->None:
    df.info() # 1

    df.describe() #2
    
    temp = df.loc[:, "UNIQUE_CARRIER"].unique()
    display(temp)
    print(len(temp))

    temp = (df.loc[:, "ORIGIN_AIRPORT_ID"] + df.loc[:, "DEST_AIRPORT_ID"]).unique() #4
    display(temp)
    print(len(temp))

    print(min(df.loc[:, "FL_DATE"]), max(df.loc[:, "FL_DATE"])) # 5
    
    

### 3 - Filter out all canceled flight

This function filters out all the canceled flight from the dataFrame

In [324]:
def filterCancelFlight(df:pd.DataFrame)->pd.DataFrame:
    return df[df.loc[:, 'CANCELLED'] < TOL].drop(columns=['CANCELLED', 'CANCELLATION_CODE'])

### 4 - QUERIES 
Use any pandas method and functionality to answer the following queries:
- how many flights had each carrier operated?
- for each carrier, compute the mean delay considering all possible reasons (due to the carrier,
weather, etc.)


In [325]:
def meanCarriersDelay(df: pd.DataFrame, vers:bool)->pd.Series:
    if vers:
        return (df.loc[:, [col for col in df.columns if 'delay' in col.strip().lower()]]
            .set_index(keys=df.loc[:, 'UNIQUE_CARRIER']).sum(axis=1, numeric_only=True)
            .groupby(['UNIQUE_CARRIER'], axis=0)
            .mean(numeric_only=True)) 
    else:
        return (df.loc[:, ['UNIQUE_CARRIER']+[col for col in df.columns if 'delay' in col.strip().lower()]]
            .groupby(['UNIQUE_CARRIER']).mean(numeric_only=True)) 


def queries(df:pd.DataFrame) -> None:
    print("Unique Carriers flight")
    ans = df.loc[:, 'UNIQUE_CARRIER'].value_counts()
    for k in ans.index:
        print(k, '\t', ans[k])
        
    print("Mean delay for carriers ")
    display(meanCarriersDelay(df, False))

### 5 - Adding columns
add two new columns to your DataFrame:
- weekday: it is the day of the week expressed as an integer number. Check out Pandas dayofweek
attribute.
- delaydelta: it is the difference between the arrival delay and the departure one.

In [326]:
def addWeekDay(df:pd.DataFrame)->None:
    df.insert(loc=1, column='DayOfWeek', value=pd.Series(pd.to_datetime(df.loc[:, "FL_DATE"].values)).dt.day_name())
    
def addDelayDelta(df: pd.DataFrame)->None:
    df.insert(loc=26, column='DelayDelta', value=df.loc[:, 'ARR_DELAY']-df.loc[:, 'DEP_DELAY'])

### 6 - Correlation between delay and day of the week
Choose one of the visualization tools that you know and inspect the arrival delay as a function of the
day of the week. Can you find any correlation?

In [327]:
def mappingFloatDelay(delay:float)->str:
    if delay < 5:
        return 'no delay'
    elif 5 <= delay <= 30:
        return 'small delay'
    elif 30 < delay < 60:
        return 'medium delay'
    else:
        return 'severe delay' 

def delayByDayOfWeek(df:pd.DataFrame, )->None:
    df['DelayCategory'] = df['ARR_DELAY'].apply(mappingFloatDelay)
    
    (df.groupby(['DayOfWeek', 'DelayCategory']).size().unstack().fillna(0) 
    .drop(labels='no delay', axis=1)
    .plot(kind='bar', stacked=False))
    
    plt.xlabel('Day of the Week')
    plt.ylabel('Count of Delays')
    plt.title('Delay Categories by Day of the Week')
    plt.legend(title='Delay Category')
    plt.show()
    
    df.drop(['DelayCategory'])    

### 7 - WeekEnd Stats

Consider the weekend days only, compute, for each carrier, the mean arrival delay. <br>
Now consider the working days and compute, for each carrier, the mean arrival time.
<br>Then, compare the delays in working days and in weekends for each company.
<br>Are you able to identify companies that are delayed only in weekends or only in working days?
Why?


In [ ]:
def meanArrivalDelayTime(df:pd.DataFrame, days:list[str], col:str)->None:
    temp = df.loc[:, ['UNIQUE_CARRIER', 'DayOfWeek', col]]
    display(temp[temp['DayOfWeek'].isin(days)].groupby(['UNIQUE_CARRIER', 'DayOfWeek']).mean(numeric_only=True).unstack())
    

def meanArrivalArrivalTime(df: pd.DataFrame, daysToAvoid: list[str])->None:
    df['DELTA_TIME'] = df.loc[:, 'ARR_TIME']-df.loc[:, 'DEP_TIME']
    temp = df.loc[:, ['UNIQUE_CARRIER', 'DayOfWeek', 'DELTA_TIME']]
    display(temp[~temp['DayOfWeek'].isin(daysToAvoid)].groupby(['UNIQUE_CARRIER', 'DayOfWeek']).mean(numeric_only=True).unstack())
    
    
def compareDelayByDayOfWeek(df: pd.DataFrame, weekEnd:list[str])->None:
    temp = df.loc[:, ['UNIQUE_CARRIER', 'DayOfWeek', 'DelayDelta']]
    temp = (temp[temp['DayOfWeek'].isin(weekEnd)].rename(columns={'DelayDelta':'WeekEndDelay'}, inplace=False).groupby(['UNIQUE_CARRIER']).agg({'WeekEndDelay':['min', 'max', 'sum', 'mean']})
    .join(+temp[~temp['DayOfWeek'].isin(weekEnd)].rename(columns={'DelayDelta':'WeekDelay'}, inplace=False).groupby(['UNIQUE_CARRIER']).agg({'WeekDelay':['min', 'max', 'sum', 'mean']})))
    display(temp)
    display(temp.agg('min', axis=0).unstack())

### 8 - Multi Index
Create a Pandas DataFrame with a multi-index composed of the columns: tunique_carrier, origin,
dest, fl_date.

In [351]:
def multiIndex(df:pd.DataFrame, indexes:list[str])->pd.DataFrame:
    return df.set_index(keys=indexes)

### 9 - Queries
For each flight operated by American Airlines (AA) and Delta Airlines (DL), taken off from the Los
Angeles International Airport (LAX) and for each date, display the departure time and delay.


In [360]:
def queriesAAandDL(df:pd.DataFrame)->None:
    print(df.index)

### MAIN FUNCTION
This is the main function of our program, it does:
<ol>
<li>Read the input flight file</li>
<li>Answers a set of questions</li>
<li>Filters out the cancelled flights</li>
<li>Answers a set of queries</li>
<li>Adds two new columns to the dataFrame</li>
</ol>

In [361]:
def main()->None:
    df = readFileInputFlight(FLIGHT_DELAY_INPUT) # 1
    #answerQuestion(df) # 2
    df = filterCancelFlight(df) # 3
    df = df.reindex(index=list(range(len(df))))
    # queries(df) # 4
    addWeekDay(df) # 5
    addDelayDelta(df) # 5
    # delayByDayOfWeek(df) # 6
    # meanArrivalDelayTime(df, ['Saturday','Sunday'], 'ARR_DELAY') # 7
    # meanArrivalArrivalTime(df, ['Saturday', 'Sunday']) # 7
    #compareDelayByDayOfWeek(df,['Saturday', 'Sunday']) # 7
    df = multiIndex(df, ["UNIQUE_CARRIER", "ORIGIN_CITY_NAME", "DEST_CITY_NAME","FL_DATE"]) # 8
    queriesAAandDL(df) #9
    
main()

MultiIndex([('AA',          'New York, NY',       'Los Angeles, CA', ...),
            ('AA',       'Los Angeles, CA',          'New York, NY', ...),
            ('AA',       'Los Angeles, CA',          'New York, NY', ...),
            ('AA', 'Dallas/Fort Worth, TX',          'Honolulu, HI', ...),
            ('AA',           'Kahului, HI', 'Dallas/Fort Worth, TX', ...),
            ('AA', 'Dallas/Fort Worth, TX',           'Kahului, HI', ...),
            ('AA',          'Honolulu, HI', 'Dallas/Fort Worth, TX', ...),
            ('AA',          'New York, NY',     'San Francisco, CA', ...),
            ('AA',       'Los Angeles, CA',          'New York, NY', ...),
            ('AA',     'San Francisco, CA',          'New York, NY', ...),
            ...
            ('DL',        'Fort Myers, FL',           'Atlanta, GA', ...),
            ('DL',             'Omaha, NE',           'Atlanta, GA', ...),
            ('DL',          'New York, NY',           'Orlando, FL', ...),
         